Libraries needed

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [19]:
# import dataset
(training_genoms, training_labels), (test_genoms, test_labels) = tfds.load(
    "genomics_ood",
    as_supervised=True, # to include the labels
    split=["train[0:100000]", "test[0:1000]"], #100.000 example for training and 1000 for testing
    batch_size=(-1) # -1 to get the full dataset as a tf.Tensor
    )



In [5]:
def onehotify(tensor):
  vocab = {'A':'1', 'C': '2', 'G':'3', 'T':'0'}
  for key in vocab.keys():
    tensor = tf.strings.regex_replace(tensor, key, vocab[key])
    split = tf.strings.bytes_split(tensor)
    labels = tf.cast(tf.strings.to_number(split), tf.uint8)
    onehot = tf.one_hot(labels, 4)
    onehot = tf.reshape(onehot, (-1,))
  return onehot


need to zip the data and lebels and run onehotify


In [24]:
# preprocessing the data

# tensorflow.python.framework.ops.EagerTensor has to attribute map
# so converting the labels into datasets using tf.data.Dataset.from_tensor_slices

# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/data.ipynb

train_data = tf.data.Dataset.from_tensor_slices(training_genoms)
train_label = tf.data.Dataset.from_tensor_slices(training_labels)


genoms_ds = train_data.map(onehotify)
#labels_ds =  ??

# put it together
train_ds = tf.data.Dataset.zip((genoms_ds, labels_ds))


# batch it

train_ds.batch()


# shuffel it
train_ds.shuffle()

# prefetch it
train_ds.prefetch()




# do the same things for training data set



Designing the Model

In [8]:
# from keras.models import Sequential
# from keras.layers import Dense , Activation


# model = Sequential()

# model.add(tf.keras.layers.Dense(256, activation=tf.nn.sigmoid))
# model.add(tf.keras.layers.Dense(256, activation=tf.nn.softmax))
# model.add(tf.keras.layers.Dense(10, activation=tf.nn.relu))

# model.compile(optimizer='sgd', loss=tf.keras.losses.CategoricalCrossentropy())

# #model.fit(training_genoms,training_labels, epochs=500)
# #model.summary(line_length=None, positions=None, print_fn=None)

class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

    def call(self, inputs):
        x = self.dense1(inputs)

        x = self.dense2(x)
        x = self.output(x)
        return x




Training, mainly taken from courseware


In [9]:
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input)
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [15]:
tf.keras.backend.clear_session()

#For showcasing we only use a subset of the training and test data (generally use all of the available data!)
train_dataset = train_dataset.take(1000)
test_dataset = test_dataset.take(100)

### Hyperparameters
num_epochs = 10
learning_rate = 0.01

# Initialize the model.
model = MyModel()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: SGD with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.SGD(learning_rate)


# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_dataset:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

AttributeError: ignored

visualization


In [ ]:
import matplotlib.pyplot as plt

# Visualize accuracy and loss for training and test data.
plt.figure()
line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Loss/Accuracy")
plt.legend((line1,line2, line3),("training","test", "test accuracy"))
plt.show()